<a href="https://colab.research.google.com/github/aashrey1234/ICDesign/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#convert MNIST image files into a Tensor of 4-dims
transform = transforms.ToTensor()


In [4]:
#Train Data
Train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 54608958.27it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1694489.90it/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 13268609.91it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 13824766.89it/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [5]:
#test Data
Test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [6]:
Test_data
Train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
train_loader = DataLoader(Train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(Test_data, batch_size=10, shuffle=True)

In [10]:
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)


In [13]:
#grab 1 MNIST record/image
for i, (x_train, y_train) in enumerate(Train_data):
  break

In [15]:
x_train.shape

torch.Size([1, 28, 28])

In [16]:
x = x_train.view(1,1,28,28)

In [17]:
x = F.relu(conv1(x))

In [19]:
x.shape

torch.Size([1, 6, 26, 26])

In [20]:
x = F.max_pool2d(x,2,2)

In [21]:
x.shape

torch.Size([1, 6, 13, 13])

In [22]:
x = F.relu(conv2(x))

In [23]:
x.shape

torch.Size([1, 16, 11, 11])

In [24]:
x = F.max_pool2d(x,2,2)

In [25]:
x.shape

torch.Size([1, 16, 5, 5])

In [26]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    #Fully connceted layer
    self.fc1 = nn.Linear(5*5*16, 128)
    self.fc2 = nn.Linear(128, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2,2)

    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2,2)

    x = x.view(-1, 16*5*5)

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.log_softmax(X,dim=1)

In [27]:
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time
start_time = time.time()

epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  for b,(x_train, y_train) in enumerate(train_loader):
     b+=1
     y_pred = model(x_train)
     loss = criterion(y_pred, y_train)

     predicted = torch.max(y_pred.data, 1)[1]
     batch_corr = (predicted == y_train).sum()
     trn_corr += batch_corr

     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     if b%600 == 0;
     print(f'Epoch: {1} Batch: {b} Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)

#test
  with torch.no_grad():
    for b,(x_test, y_test) in enumerate(test_loader):
      y_val = model(x_test)
      predicted = torch.max(y_val.dat, 1)[1]
      tst_corr += (predicted == y_test).sum()

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

current_time = tiem.time()
total = current_time - start_time
print(f'Training took: {total/60} minutes')